In [1]:
# gives me 4 range that partition number range from 0 to max into 4 equal parts
def partition(max, n):
    range_ = []
    for i in range(n):
        
        range_.append(round(i*max/n, 4))
    return range_
partition(0.2, 2)


[0.0, 0.1]

In [4]:
import torch
l = [0.05,0.1,0.2]

for i in range(len(l)):
    for j in range(i+1, len(l)):
        print(l[i], l[j])
my_list = [1, 2, 3, 4, 5]
my_tensor = torch.tensor(my_list)
print(my_tensor)


0.05 0.1
0.05 0.2
0.1 0.2


In [1]:
from global_seq_align_gpu import global_alignment_batch_no_gap_penalty
from batchify import get_batches
from utils import load_sent
from vocab_less import Vocab
import torch
batch_size = 100000

sents = load_sent('/home/amos/MgDB_amos/data/virus_all.fasta_noAmbiguous_cdhit_onebase_90percent')
vocab = Vocab()
batches= get_batches(sents, vocab, batch_size, device = torch.device("cuda"))

/home/amos/miniconda3/envs/py38/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Time taken: 0.2551767826080322 seconds


In [2]:
# read file line by line
from tqdm import tqdm
with open('/home/amos/MgDB_amos/data/virus_all.fasta_noAmbiguous_cdhit_onebase_10percent_evenly_repeat100_testing') as f:    
    lines = f.readlines()
    every_100th_line = lines[::100]
    min_score = 64
    for line in every_100th_line:
        
        for batch in tqdm(batches):
            
            seq1 = batch[1:].permute(1, 0)
            batch_size = seq1.size(0)
            seq2 = torch.tensor([[vocab.word2idx[c] for c in line.strip() if c != ' '] for i in range(batch_size)], dtype=torch.long, device='cuda')
            score = global_alignment_batch_no_gap_penalty(seq1, seq2)
            score_current_min = torch.min(score).item()
            # break
            # print(score_current_min)
            if score_current_min < min_score:
                min_score = score_current_min


 20%|██        | 1/5 [00:02<00:11,  2.82s/it]


KeyboardInterrupt: 